In [13]:
import time
import json
import numpy as np
import pandas as pd
import pandas_datareader.data as pdweb

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300
plt.rcParams['figure.figsize'] = (15, 8)
plt.style.use('seaborn-white')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
path = r'C:\Users\tarek.vilela.CVER\Documents\chromedriver.exe'

In [2]:
credentials = {'login': '<USUARIO>', 'password': '<SENHA>'} # formato da senha '123456'

In [3]:
def abrir():
    global driver
    driver = webdriver.Chrome(path)
    driver.maximize_window()
    driver.get('https://www.rico.com.vc/login/')

In [4]:
def login():
    global driver
    time.sleep(5)
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[2]/form/div/div[1]/div/div/input').send_keys(credentials['login'])
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[2]/form/div/div[2]/button').click()
    def enter_password(key):
        global driver
        pass_btns = {}
        for i in range(1, 6):
            pass_btns[i] = \
            list(driver.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[2]/div/form/div/div[%d]' % i)\
                 .text.replace('\nou\n', ''))
        for i in pass_btns:
            if key in pass_btns[i]:
                driver.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[2]/div/form/div/div[%d]' % i).click()
    
    for p in list(credentials['password']):
        enter_password(p)
        time.sleep(0.5)

    driver.find_element_by_xpath('//*[@id="app"]/div/div[2]/div[1]/div/div[2]/div/form/button').click()
    time.sleep(8)
    driver.find_element_by_xpath('/html/body/div[6]/div/div[5]/a[1]').click()

In [5]:
def get_allocated_value():
    global driver
    driver.get('https://www.rico.com.vc/dashboard/acoes/')
    time.sleep(5)
    tabela_ = driver.find_element_by_xpath('//*[@id="tableAllocatedValue"]')
    linhas = tabela_.find_elements_by_tag_name('tr')

    tabela = []
    for linha in linhas[2:]:
        try:
            x = [linha.find_elements_by_tag_name('td')[0].text, 
                 float(linha.find_elements_by_tag_name('td')[1].text.split(' ')[1].replace(',', '.')),
                 int(linha.find_elements_by_tag_name('td')[2].text)]
            tabela.append(x)
        except:
            pass
    
    return pd.DataFrame(tabela, columns = ['Ticker', 'Preço Atual', 'Quantidade'])

In [6]:
def get_proventos():
    global driver
    driver.get('https://www.rico.com.vc/dashboard/acoes/proventos/')
    time.sleep(5)
    frame = driver.find_element_by_xpath('//*[@id="iframe-proceeds"]')
    driver.switch_to.frame(frame)
    
    driver.find_element_by_xpath('//*[@id="ctl00_MainContent_uscProfit_txtBeginDate"]').clear()
    driver.find_element_by_xpath('//*[@id="ctl00_MainContent_uscProfit_txtBeginDate"]').send_keys('01/01/2019')
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ctl00_MainContent_uscProfit_imgSearch"]').click()
    
    tabela = []
    paginas = driver.find_element_by_xpath('//*[@id="ctl00_MainContent_uscProfit_grdProfit"]/tbody/tr[22]/td/table')\
                    .find_elements_by_tag_name('td')
    
    for pag in range(1, len(paginas)+1):
        try:
            driver.find_element_by_xpath('//*[@id="ctl00_MainContent_uscProfit_grdProfit"]/tbody/tr[22]/td/table/tbody/tr/td[%d]/a' % pag).click()
        except:
            pass
        
        time.sleep(2)
        tabela_ = driver.find_element_by_xpath('//*[@id="ctl00_MainContent_uscProfit_grdProfit"]')
        linhas = tabela_.find_elements_by_tag_name('tr')
        
        for linha in linhas[1:-2]:
                x = [int(linha.find_elements_by_tag_name('td')[0].text),
                         linha.find_elements_by_tag_name('td')[1].text,
                         int(linha.find_elements_by_tag_name('td')[2].text),
                         float(linha.find_elements_by_tag_name('td')[3].text.replace(',', '.')),
                         float(linha.find_elements_by_tag_name('td')[4].text.replace(',', '.')),
                         float(linha.find_elements_by_tag_name('td')[5].text.replace(',', '.')),
                         linha.find_elements_by_tag_name('td')[6].text]
                tabela.append(x)
                
    df = pd.DataFrame(tabela, columns = ['Tipo', 'Ticker', 'Quantidade', 'Valor Bruto', 'IR', 'Valor Líquido', 'Data'])
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
    
    x = []
    for i in range(df.shape[0]):
        try:
            x.append(pdweb.DataReader(df['Ticker'].iloc[i] + '.SA', 
                               start=df['Data'].iloc[i], 
                               end=df['Data'].iloc[i], 
                               data_source='yahoo')['Close'].iloc[0])
        except:
            x.append(np.nan)
    
    df['Preço_data'] = x
    provs = df[(df['Tipo']==14) & (df['Valor Bruto'] > 0)]
    amort = df[(df['Tipo']==17) & (df['Valor Bruto'] > 0)]
    subs = df[(df['Tipo']==50) & (df['Valor Bruto'] > 0)]
    
    return provs, amort, subs

In [7]:
def get_extrato():
    global driver
    driver.get('https://www.rico.com.vc/dashboard/conta/extrato/')
    time.sleep(5)
    driver.find_element_by_xpath('/html/body/section/div/div[2]/div/div/section/div/form/div/div[1]/div/input').clear()
    driver.find_element_by_xpath('/html/body/section/div/div[2]/div/div/section/div/form/div/div[1]/div/input').send_keys('01/01/2019')
    time.sleep(0.5)
    driver.find_element_by_xpath('/html/body/section/div/div[2]/div/div/section/div/form/div/div[1]/div/input').send_keys(Keys.RETURN)
    
    tabela_ = driver.find_element_by_xpath('/html/body/section/div/div[2]/div/div/section/div/table')
    linhas = tabela_.find_elements_by_tag_name('tr')
    
    tabela = []
    for linha in linhas[1:]:
        x = [linha.find_elements_by_tag_name('td')[0].text,
             linha.find_elements_by_tag_name('td')[1].text,
             linha.find_elements_by_tag_name('td')[2].text,
             float(linha.find_elements_by_tag_name('td')[3].text.split(' ')[1].replace('.', '').replace(',', '.')),
             float(linha.find_elements_by_tag_name('td')[4].text.split(' ')[1].replace('.', '').replace(',', '.'))]
        tabela.append(x)
        
    df = pd.DataFrame(tabela, columns = ['Liquidação', 'Lançamento', 'Transação', 'Valor', 'Saldo'])
    df['Liquidação'] = pd.to_datetime(df['Liquidação'], format='%d/%m/%Y')
    
    return df

In [8]:
def get_saldo():
    global driver
    driver.find_element_by_xpath('//*[@id="new-dashboard"]/aside/div[2]/div/section/h2').click()
    saldo = float(driver.find_element_by_xpath('//*[@id="new-dashboard"]/aside/div[2]/div/section/dl/dd[4]/strong')\
                            .text.split(' ')[1].replace(',', '.'))
    
    driver.find_element_by_xpath('//*[@id="new-dashboard"]/h2').click()
    investido = float(driver.find_element_by_xpath('//*[@id="new-dashboard"]/div/div[1]/div/strong[2]').text[3:].replace('.', '').replace(',', '.'))
    
    return saldo, investido

In [9]:
def load_data():
    with open('info.txt', 'r') as f:
        dados = json.load(f)
    
    ativos = pd.DataFrame(json.loads(dados['ativos']))
    
    extrato = pd.DataFrame(json.loads(dados['extrato']))
    extrato['Liquidação'] = pd.to_datetime(extrato['Liquidação'])
    
    provs = pd.DataFrame(json.loads(dados['provs']))
    provs['Data'] = pd.to_datetime(provs['Data'])
    
    amort = pd.DataFrame(json.loads(dados['amort']))
    amort['Data'] = pd.to_datetime(amort['Data'])
    
    subs = pd.DataFrame(json.loads(dados['subs']))
    subs['Data'] = pd.to_datetime(subs['Data'])
    
    return dados['saldo'], dados['valor_investido'], ativos, extrato, provs, amort, subs, dados['TEDs']

In [10]:
def save_data(conta):
    ativos = conta.ativos
    
    extrato = conta.extrato
    extrato['Liquidação'] = extrato['Liquidação'].astype(str)
    
    provs = conta.provs
    provs['Data'] = provs['Data'].astype(str)
    
    amort = conta.amort
    amort['Data'] = amort['Data'].astype(str)
    
    subs = conta.subs
    subs['Data'] = subs['Data'].astype(str)
    
    info = {'saldo': conta.saldo, 
        'valor_investido': conta.valor_investido, 
        'ativos': ativos.to_json(), 
        'extrato': extrato.to_json(), 
        'provs': provs.to_json(),
        'amort': amort.to_json(), 
        'subs': subs.to_json(), 
        'TEDs': conta.TEDs
       }
    
    with open('info.txt', 'w') as f:
        f.write(json.dumps(info))

In [11]:
class Rico:
    def __init__(self):
        self.saldo,\
        self.valor_investido,\
        self.ativos,\
        self.extrato,\
        self.provs,\
        self.amort,\
        self.subs,\
        self.TEDs = load_data()
        
    def update(self):
        abrir(); login()
        self.saldo, \
        self.valor_investido = get_saldo()
        
        self.ativos = get_allocated_value()
        
        self.provs,\
        self.amort,\
        self.subs = get_proventos()
        
        self.extrato = get_extrato()
        
        self.TEDs = self.extrato[self.extrato['Transação']=='TED - RECEBIMENTO DE TED - SPB']['Valor'].sum()
        driver.quit()
        save_data(self)

In [12]:
conta = Rico()

In [ ]:
conta.update()